In [3]:
import pandas as pd
import datetime as dt
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import operator
import numpy as np

/Users/yishh/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [5]:
G = nx.read_edgelist("../0_cleaning/max_wcc_graph.edgelist", create_using=nx.DiGraph(), delimiter="\t", data=True)

In [13]:
# post_2005_edges = pd.read_csv("../0_cleaning/max_wcc_graph.edgelist", low_memory=False)

In [30]:
dates = nx.get_edge_attributes(G, 'date')
# weights = nx.get_edge_attributes(G, 'weight')

flat_table = map(lambda x: [x[0], x[1], dates[x]] , dates)
wcc_df = pd.DataFrame(flat_table)
wcc_df.columns = ["investor", "company", "invest_date"]
wcc_df["invest_date"].apply(pd.to_datetime)
post_2005_edges = wcc_df

In [24]:
total_periods = 132
rng = pd.date_range('1/1/2005', periods= total_periods, freq='M')
months_per_period = 3
for i in range(1,total_periods/months_per_period):
    period = i * months_per_period
    curr_rng = rng[:period]
    curr_start = curr_rng[0]
    curr_end = curr_rng[-1]
    print curr_start
    mask = (post_2005_edges['invest_date'] >= curr_start) & (post_2005_edges['invest_date'] <= curr_end)
    period_snapshot = post_2005_edges[mask]
    file_name = "../../unweighted_wcc_snapshots/cumulative/enddate_" + str(curr_end)[:10] + ".csv"
    period_snapshot.to_csv(file_name, index='false')

2005-01-31 00:00:00


TypeError: Cannot compare type 'Timestamp' with type 'str'

In [26]:
total_periods = 132
rng = pd.date_range('1/1/2005', periods= total_periods, freq='M')
months_per_period = 3
for i in range(1,total_periods/months_per_period):
    period = i * months_per_period
    curr_rng = rng[period-months_per_period:period]
    curr_start = curr_rng[0]
    curr_end = curr_rng[-1]
#     print curr_rng 
#     print curr_start, curr_end
    
    # post_2005_edges 
    mask = (post_2005_edges['invest_date'] >= curr_start) & (post_2005_edges['funded_at'] <= curr_end)
    period_snapshot = post_2005_edges[mask]
    file_name = "../../unweighted_wcc_snapshots/diff/enddate_" + str(curr_end)[:10] + ".csv"
#     print file_name
    period_snapshot.to_csv(file_name, index='false')
#     if i ==3:
#         break


TypeError: Cannot compare type 'Timestamp' with type 'str'

In [85]:
#calculate all shortest path lengths
# G = nx.path_graph(5)
all_path_lengths = nx.all_pairs_shortest_path_length(G)
all_path_lengths_sorted = []
for i in range(len(all_path_lengths)):
    p = all_path_lengths[i]
    curr_distances =  sorted(list(zip(p.keys(),p.values())), key = lambda x: x[1], reverse = True)
    all_path_lengths_sorted.append(curr_distances)
    
all_path_lengths_sorted

[[(4, 4), (3, 3), (2, 2), (1, 1), (0, 0)],
 [(4, 3), (3, 2), (0, 1), (2, 1), (1, 0)],
 [(0, 2), (4, 2), (1, 1), (3, 1), (2, 0)],
 [(0, 3), (1, 2), (2, 1), (4, 1), (3, 0)],
 [(0, 4), (1, 3), (2, 2), (3, 1), (4, 0)]]

In [86]:
#calculate number of paths between every other node
# G = nx.path_graph(5)
g_size = len(G)

all_number_paths = np.zeros((g_size,g_size))

for i in range(g_size):
    for j in range(i,g_size):
        if i == j:
            continue
        all_paths = nx.all_simple_paths(G, source=i, target=j)
        num = len(list(all_paths))
        all_number_paths[i,j] = num
        all_number_paths[j,i] = num

all_number_paths


array([[ 0.,  1.,  1.,  1.,  1.],
       [ 1.,  0.,  1.,  1.,  1.],
       [ 1.,  1.,  0.,  1.,  1.],
       [ 1.,  1.,  1.,  0.,  1.],
       [ 1.,  1.,  1.,  1.,  0.]])

In [89]:
#calculate number of paths between every other node
# G = nx.path_graph(5)
g_size = len(G)

all_number_paths = np.zeros((g_size,g_size))

adamic_index = nx.adamic_adar_index(G)
adamic_index_list = [p for p in adamic_index]
adamic_index_list


[(0, 2, 1.4426950408889634),
 (0, 3, 0),
 (0, 4, 0),
 (1, 3, 1.4426950408889634),
 (1, 4, 0),
 (2, 4, 1.4426950408889634)]

In [91]:
#once i have generated every graph
#for each snapshot
    #generate a graph
    #for each graph
        #get all shortest paths lengths
        #get number paths between every node
        #get adamic index
        #store it properly
        
#random link predictor


In [8]:
G

In [ ]:
post_2005_edges

In [23]:
wcc_df

,investor,company,invest_date
0,/organization/mayfield-fund,/organization/cloudvelocity,2015-02-12
1,/organization/foundation-capital,/organization/dogvacay,2014-11-10
2,/organization/blue-ant-investments,/organization/hiline-coffee-company,2014-01-26
3,/organization/borealis-ventures,/organization/spaceclaim,2010-09-15
4,/organization/horizon-group,/organization/geosentric,2007-09-01
5,/organization/universal-music-group,/organization/doppler-labs,2015-07-07
6,/organization/industrifonden,/organization/footway,2012-01-25
7,/person/david-ulevitch,/organization/thirdlove,2013-08-01
8,/organization/needham-capital-partners,/organization/chipx,2008-07-22
9,/organization/radius-ventures,/organization/endogastric-solutions,2014-05-06
